In [1]:
import numpy as np
from scipy.stats import norm

class VanillaOption:
    def __init__(self, S, K, T, r, sigma, option_type):
        self.S = S
        self.K = K
        self.T = T
        self.r = r
        self.sigma = sigma
        self.option_type = option_type
        if self.option_type not in ["Call", "Put"]:
            raise ValueError("option_type must be 'Call' or 'Put'")

    def d1(self):
        return (np.log(self.S / self.K) + (self.r + 0.5 * self.sigma ** 2) * self.T) / (self.sigma * np.sqrt(self.T))

    def d2(self):
        return self.d1() - self.sigma * np.sqrt(self.T)

    def price(self):
        d1, d2 = self.d1(), self.d2()
        if self.option_type == "Call":
            return self.S * norm.cdf(d1) - self.K * np.exp(-self.r * self.T) * norm.cdf(d2)
        else:
            return self.K * np.exp(-self.r * self.T) * norm.cdf(-d2) - self.S * norm.cdf(-d1)

    def delta(self):
        d1 = self.d1()
        if self.option_type == "Call":
            return norm.cdf(d1)
        else:
            return norm.cdf(d1) - 1

    def gamma(self):
        d1 = self.d1()
        return norm.pdf(d1) / (self.S * self.sigma * np.sqrt(self.T))

    def vega(self):
        d1 = self.d1()
        return self.S * norm.pdf(d1) * np.sqrt(self.T) / 100  # En %

    def theta(self):
        d1, d2 = self.d1(), self.d2()
        term1 = -self.S * norm.pdf(d1) * self.sigma / (2 * np.sqrt(self.T))
        if self.option_type == "Call":
            term2 = self.r * self.K * np.exp(-self.r * self.T) * norm.cdf(d2)
            return (term1 - term2) / 365
        else:
            term2 = self.r * self.K * np.exp(-self.r * self.T) * norm.cdf(-d2)
            return (term1 + term2) / 365

    def rho(self):
        d2 = self.d2()
        if self.option_type == "Call":
            return self.K * self.T * np.exp(-self.r * self.T) * norm.cdf(d2) / 100
        else:
            return -self.K * self.T * np.exp(-self.r * self.T) * norm.cdf(-d2) / 100


In [2]:
option = VanillaOption(S=206, K=98, T=0.8, r=0.05, sigma=0.2, option_type='Call')
print("Prix :", option.price())
print("Delta :", option.delta())
print("Gamma :", option.gamma())

Prix : 111.8426662646066
Delta : 0.9999960158774829
Gamma : 5.050841057062049e-07


In [3]:
import ipywidgets as widgets
from IPython.display import display, HTML

# === Sliders & Dropdown ===
style = {'description_width': '100px'}
layout = widgets.Layout(width='350px', margin='0 0 10px 0')

S_slider = widgets.FloatSlider(value=100, min=10, max=1000, step=1, style=style, layout=layout)
K_slider = widgets.FloatSlider(value=100, min=10, max=1000, step=1, style=style, layout=layout)
T_slider = widgets.FloatSlider(value=1, min=0.01, max=2, step=0.01, style=style, layout=layout)
r_slider = widgets.FloatSlider(value=0.05, min=0, max=0.1, step=0.001, style=style, layout=layout)
sigma_slider = widgets.FloatSlider(value=0.2, min=0.01, max=1.0, step=0.01, style=style, layout=layout)
option_type_dropdown = widgets.Dropdown(options=['Call', 'Put'], value='Call', style=style, layout=layout)

# === Paramètres UI ===
param_ui = widgets.VBox([
    widgets.HTML("<b style='color:#2a4365;'>Option parameters</b>"),
    widgets.HTML("<b>Spot - S</b>"), S_slider,
    widgets.HTML("<b>Strike - K</b>"), K_slider,
    widgets.HTML("<b>Maturity - T</b>"), T_slider,
    widgets.HTML("<b>Risk-free interest rate - r</b>"), r_slider,
    widgets.HTML("<b>Volatility - σ</b>"), sigma_slider,
    widgets.HTML("<b>Type</b>"), option_type_dropdown
])
param_ui.layout = widgets.Layout(
    min_height='400px',
    border='1px solid #ccc',
    padding='15px',
    border_radius='10px',
    background_color='#ffffff',
    width='400px',
    margin='30px 30px 30px 30px'
)

# === Boîte pour les résultats ===
result_box = widgets.Output()
result_box.layout = widgets.Layout(
    width='400px',
    min_width='350px',
    border='1px solid #ccc',
    padding='20px',
    border_radius='10px',
    background_color='#f4f6f9',
    margin='30px 30px 30px 0px'
)

# === Fonction de mise à jour ===
def update_price(S, K, T, r, sigma, option_type):
    with result_box:
        result_box.clear_output(wait=True)
        opt = VanillaOption(S, K, T, r, sigma, option_type)
        display(HTML(f"""
        <div style="font-family: Arial; color: #333;">
            <h4 style="margin-top: 0; color: #2a4365;">Results – Option <b>{option_type.capitalize()}</b></h4>
            <table style="width: 100%; font-size: 14px;">
                <tr><td><b>Prix :</b></td><td>{opt.price():.4f}</td></tr>
                <tr><td><b>Delta :</b></td><td>{opt.delta():.4f}</td></tr>
                <tr><td><b>Gamma :</b></td><td>{opt.gamma():.4f}</td></tr>
                <tr><td><b>Vega :</b></td><td>{opt.vega():.4f}</td></tr>
                <tr><td><b>Theta :</b></td><td>{opt.theta():.4f}</td></tr>
                <tr><td><b>Rho :</b></td><td>{opt.rho():.4f}</td></tr>
            </table>
        </div>
        """))

# === Affichage dynamique ===
out = widgets.interactive_output(update_price, {
    'S': S_slider, 'K': K_slider, 'T': T_slider,
    'r': r_slider, 'sigma': sigma_slider, 'option_type': option_type_dropdown
})

# === Contenu : paramètres + résultats côte à côte ===
content = widgets.HBox([param_ui, result_box], layout=widgets.Layout(
    justify_content='center',
    align_items='flex-start',
    width='100%',
    gap='60px',
    padding='0px 60px'
))

# === Interface complète avec titre ===
full_ui = widgets.VBox([
    widgets.HTML("<h2 style='text-align:center; color:#2a4365; text-decoration: underline; margin-bottom:30px;'> Vanilla Option Pricer – Black-Scholes Model</h2>"),
    content
], layout=widgets.Layout(
    width='100%',
    background_color="#ffffff"
))

from IPython.display import display, HTML

# Appliquer un style général (thème doux)
display(HTML("""
<style>
    .widget-label {
        color: #333 !important;
        font-weight: bold;
    }
    .widget-slider > .widget-readout {
        color: #333;
    }
    .widget-box, .widget-hbox, .widget-vbox {
        background-color: #f0f4f8 !important;
        border-radius: 10px;
        padding: 15px;
        border: 1px solid #ccc;
    }
    .output_area {
        background-color: #ffffff;
        border: 1px solid #ddd;
        border-radius: 10px;
        padding: 10px;
    }
    h2 {
        font-family: Arial, sans-serif;
        color: #2a4365;
        text-decoration: underline;
    }
</style>
"""))

display(full_ui, out)


Output()